DATA 620 Final Project

In [2]:
import pandas as pd

In [3]:
pd.read_parquet('training_data_raw.parquet')

,precinct,prop_code,voter_choice,doc_name,ability,able,abolishing,abortion,absent,abuse,...,computer_no_computer,internet_fixed_broadband,internet_dial_up_or_cellular,internet_no_internet,age_18_to_24,age_25_to_34,age_35_to_44,age_45_to_54,age_55_to_64,age_65+
0,0050003A,PROP03_2018,opposed,PROP03_2018,0.0,0.0,0.0,0.0,0.0,0.0,...,0.082346,0.794703,0.124973,0.080324,0.141587,0.127311,0.124503,0.158905,0.249941,0.197753
1,0050003A,PROP03_2018,opposed,PROP03_2018,0.0,0.0,0.0,0.0,0.0,0.0,...,0.022414,0.824363,0.175637,0.000000,0.092601,0.081967,0.143553,0.188303,0.280904,0.212672
2,0050003A,PROP03_2018,opposed,PROP03_2018,0.0,0.0,0.0,0.0,0.0,0.0,...,0.082346,0.794703,0.124973,0.080324,0.141587,0.127311,0.124503,0.158905,0.249941,0.197753
3,0050003A,PROP03_2018,opposed,PROP03_2018,0.0,0.0,0.0,0.0,0.0,0.0,...,0.022414,0.824363,0.175637,0.000000,0.092601,0.081967,0.143553,0.188303,0.280904,0.212672
4,0050003A,PROP03_2018,opposed,PROP03_2018,0.0,0.0,0.0,0.0,0.0,0.0,...,0.082346,0.794703,0.124973,0.080324,0.141587,0.127311,0.124503,0.158905,0.249941,0.197753
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2354799,9008296A,PROP12_2018,support,PROP12_2018,0.0,0.0,0.0,0.0,0.0,0.0,...,0.054778,0.608822,0.348036,0.043142,0.168248,0.173240,0.283075,0.125312,0.143285,0.106840
2354800,9008296A,PROP12_2018,support,PROP12_2018,0.0,0.0,0.0,0.0,0.0,0.0,...,0.009119,0.596512,0.386050,0.017438,0.125139,0.213517,0.236168,0.214259,0.103231,0.107687
2354801,9008296A,PROP12_2018,support,PROP12_2018,0.0,0.0,0.0,0.0,0.0,0.0,...,0.032324,0.658075,0.311415,0.030510,0.147934,0.269602,0.188876,0.188876,0.120896,0.083816
2354802,9008296A,PROP12_2018,support,PROP12_2018,0.0,0.0,0.0,0.0,0.0,0.0,...,0.054778,0.608822,0.348036,0.043142,0.168248,0.173240,0.283075,0.125312,0.143285,0.106840
